This example walks through creating and using an nnext index for nearest neighbor search. It illustrates a few
test cases to demonstrate the capabilities out of the box, and includes a visualization. To follow along with this example
in your own environment, install the pynnext package (see the README), and install the additional dependencies for
the example declared in the `requirements.txt` file in this folder.

In [ ]:
# External imports
import numpy as np

# Internal imports
import nnext

In [ ]:
# Create and initialize a client for a local running single-node server.
nnclient = nnext.Client(
    nodes=[
    {'host': 'localhost', 'port': '6040'}
  ])

In [ ]:
# Vector dimension of interest
n_dim = 768

# Create an vector index.
nnindex = nnclient.index.create(
    d=n_dim,
    name='test_index')

# Insert vectors into the index.
n_vecs = 1000
vectors = np.random.rand(n_vecs, n_dim)
nnindex.add(vectors)

# # Access our previously create index
# nnindex = nnclient.index["test_index_2"]

# Create a query vector set.
n_queries = 10
q_vectors = np.random.rand(n_queries, n_dim)

# Search for the k nearest neighbors of each vector
# in the query set, among the indexed vectors.
k = 5
_idx, _res = nnindex.search(q_vectors, k, return_vector=True)

# The search operation returns a 2d list of the indices of the nearest neighbors
# for each vector in the query set (i.e. shape (n_queries, k)), and optionally the data associated with
# the neighbor vectors themselves, i.e. nested list of shape (n_queries, k, n_dim), where entry (i, j, :) is the (n_dim)-dimensional
# vector corresponding to the jth nearest neighbor of query vector i
assert len(_idx) == n_queries
assert len(_idx[0]) == k
assert len(_res) == n_queries
assert len(_res[0]) == k
assert len(_res[0][0]) == n_dim

In [ ]:
# TODO: Figure out why server is not allowing creation of an index for 3d-vectors
# n_dim = 3

# # Create an vector index.
# nnindex = nnclient.index.create(
#     d=n_dim,
#     name='test_4d2')

# vec1 = np.array[1, 0, 0]

In [ ]:
# Sanity Check:
# TODO: Add test cases of this sort to server code

n_dim = 128

# Create an vector index.
nnindex = nnclient.index.create(
    d=n_dim,
    name='sanity_index_13')

# Create a query vector set.
n_queries = 10
q_vectors = np.random.rand(n_queries, n_dim)

# Also insert that set into the index
nnindex.add(q_vectors)

# Search for the k nearest neighbors of each vector
# in the query set, among the indexed vectors.
k = 5
_idx, _res = nnindex.search(q_vectors, k, return_vector=True)

# Ensure that:
# 1. The index of the nearest vector to each query is the "index" of the query vector (i.e. it's index in the list from which we inserted)
# 2. The actual vector asccociated with each nearest neighbor should be the query vector itself. We'll test this via vector norm,
# since equality of vectors is sensitive to storage reltated rounding errors.
for idx, q in enumerate(q_vectors):
    nn = _res[idx][0] # supposed nearest neighbor
    nn_idx = int(_idx[idx][0]) # supposed idx of that neighbor

    # Test 1
    assert nn_idx == idx

    # Test 2
    eps = 1e-6
    assert np.linalg.norm(np.array(nn) - q) <= eps